In [1]:
from __future__ import print_function
import argparse
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from data import CTDataset
from detector import PointNetDenseCls4D
import torch.nn.functional as F

batch_size = 1
workers = 4
threshold_min = 1700
threshold_max = 2700
npoints = 25000

test_dataset = CTDataset(root='../data',
                         threshold_min=int(threshold_min),
                         threshold_max=int(threshold_max),
                         npoints=npoints,
                         train=False, dim4=True)

print("# of  testing examples: {0}".format(len(test_dataset)))
num_classes = test_dataset.nclasses
print("# of    object classes: {0}".format(num_classes))

testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                             shuffle=True, num_workers=int(workers))

# of  testing examples: 200
# of    object classes: 2


In [ ]:
model = "./pth/model_99.pth"

classifier = PointNetDenseCls4D(num_points=npoints, num_classes=num_classes)
classifier.load_state_dict(torch.load(model))
classifier.cuda()
None

In [ ]:
i, data = next(enumerate(testdataloader, 0))
points, target, centroid, index = data
points, target = Variable(points), Variable(target)
points = points.transpose(2, 1)
points, target = points.cuda(), target.cuda()
pred, _ = classifier(points)
pred = pred.view(-1, num_classes)
target = target.view(-1)

loss = F.nll_loss(pred, target)
pred_choice = pred.data.max(1)[1]
correct = pred_choice.eq(target.data).cpu().sum()
print("test loss: {0} accuracy: {1}".format(
    loss.item(),
    correct.item() / float(batch_size * npoints)
))

In [ ]:
import visualize

data = points[0, :, :].transpose(1, 0).cpu().numpy()
prediction = pred_choice.view(batch_size, -1)[0, :].cpu().numpy()
print(prediction.sum())
visualize.scatter_with_target(data, prediction)

In [ ]:
gt = target.view(batch_size, -1)[0, :].cpu().numpy()
visualize.scatter_with_target(data, gt)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

confidence = pred.data.cpu().numpy()[0:npoints, :]

plt.figure()
plt.hist(confidence[:, 0], bins='auto')
plt.figure()
plt.hist(confidence[:, 1], bins='auto')

intensities = confidence[:, 1]
intensities = (intensities - intensities.min()) / (intensities.max() - intensities.min())
visualize.scatter_with_intensities(data, intensities)

In [ ]:
from utils import region_grow
seed_idx = np.argmax(confidence[:, 1])
print(seed_idx)
seed = tuple(data[seed_idx][0:3].astype(np.int))
print(seed)

seed_mask = np.zeros(npoints)
seed_mask[seed_idx] = 1
visualize.scatter_with_target(data, seed_mask)

In [ ]:
_, _, volume, _, _, _ = test_dataset.load(index[0])

seg = region_grow(volume, seed, 1)

result = np.argwhere(seg == True)

visualize.scatter(result)

In [2]:
# quantitative evaluation
import numpy as np
from IPython.core.display import clear_output

predictions = np.zeros((0, npoints))
targets = np.zeros((0, npoints))
for i, data in enumerate(testdataloader, 0):
    clear_output()
    print("[{0}/{1}]".format(i+1, len(testdataloader)))
    points, target, centroid, index = data
    points, target = Variable(points), Variable(target)
    points = points.transpose(2, 1)
    points, target = points.cuda(), target.cuda()
    #pred, _ = classifier(points)
    #pred = pred.view(-1, num_classes)
    target = target.view(-1)

    #loss = F.nll_loss(pred, target)
    #pred_choice = pred.data.max(1)[1]
    
    #prediction = pred_choice.view(batch_size, -1).data.cpu().numpy()
    prediction = np.random.randint(2, size=(batch_size, npoints))
    target = target.view(batch_size, -1).data.cpu().numpy()
    
    predictions = np.append(predictions, prediction, axis=0)
    targets = np.append(targets, target, axis=0)

[4/200]


Process Process-1:
Traceback (most recent call last):
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/py

Traceback (most recent call last):
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-0f5939dd60de>", line 7, in <module>
    for i, data in enumerate(testdataloader, 0):
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 280, in __next__
    idx, batch = self._get_batch()
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 259, in _get_batch
    return self.data_queue.get()
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/visinf/hom

KeyboardInterrupt: 

Process Process-4:
Traceback (most recent call last):
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/visinf/home/alin/miniconda3/envs/thesis/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/visinf/home/alin/pointnetdetection3d/detector/data.py", line 234, in __getitem__
    sampled, target, volume, centroid, labels, num_labels = self.load(index)
  File "/visinf/home/alin/pointnetdetection3d/detector/data.py", line 73, in load
    image = itk.imre

In [13]:
import utils
# quantitative evaluation
import numpy as np
from IPython.core.display import clear_output

predictions = np.zeros((0, npoints))
targets = np.zeros((0, npoints))
for i in range(len(test_dataset)):
    clear_output()
    print("[{0}/{1}]".format(i+1, len(test_dataset)))
    sampled, target, volume, _, _, _ = test_dataset.load(i)
    
    intensities = np.array(list(map(lambda d: volume[d[0], d[1], d[2]], sampled)))
    points = np.concatenate((sampled, np.reshape(intensities, (-1, 1))), axis=1)
    
    seed, f = utils.select_seed(points)
    seg = utils.region_grow(volume, seed, 1)
    prediction = seg[sampled[:, 0], sampled[:, 1], sampled[:, 2]].astype(np.int)
    
    predictions = np.append(predictions, np.reshape(prediction, (1, -1)), axis=0)
    targets = np.append(targets, np.reshape(target, (1, -1)), axis=0)

[158/200]


KeyboardInterrupt: 

In [14]:
import importlib
import evaluation
importlib.reload(evaluation)

mean_ap25 = evaluation.mean_average_precision(predictions, targets, iou_threshold=0.25)
mean_ap50 = evaluation.mean_average_precision(predictions, targets, iou_threshold=0.5)
mean_ap75 = evaluation.mean_average_precision(predictions, targets, iou_threshold=0.75)
average_iou = evaluation.average_iou(predictions, targets)

print("  mean_ap25: {0}".format(mean_ap25))
print("  mean_ap50: {0}".format(mean_ap50))
print("  mean_ap75: {0}".format(mean_ap75))
print("average_iou: {0}".format(average_iou))

### Random @ 10k
## Training Data:
#   mean_ap25: 0.0684
#   mean_ap50: 0.0000
#   mean_ap75: 0.0000
# average_iou: 0.1096
## Testing Data:
#   mean_ap25: 0.0900
#   mean_ap50: 0.0000
#   mean_ap75: 0.0000
# average_iou: 0.1103

### Random @ 25k
## Training Data:
#   mean_ap25: 0.0703
#   mean_ap50: 0.0000
#   mean_ap75: 0.0000
# average_iou: 0.1097
## Testing Data:
#   mean_ap25: 0.0900
#   mean_ap50: 0.0000
#   mean_ap75: 0.0000
# average_iou: 0.1104

### Random @ 50k
## Training Data:
#   mean_ap25: 0.0703
#   mean_ap50: 0.0000
#   mean_ap75: 0.0000
# average_iou: 0.1097
## Testing Data:
#   mean_ap25: 0.0900
#   mean_ap50: 0.0000
#   mean_ap75: 0.0000
# average_iou: 0.1103


### PointNet 4D Segmentation @ 10k
## Training Data:
#   mean_ap25: 0.2246
#   mean_ap50: 0.0859
#   mean_ap75: 0.0547
# average_iou: 0.1598
## Testing Data:
#   mean_ap25: 0.1950
#   mean_ap50: 0.1000
#   mean_ap75: 0.0800
# average_iou: 0.1624

### PointNet 4D Segmentation @ 25k
## Training Data:
#   mean_ap25: 0.2324
#   mean_ap50: 0.0781
#   mean_ap75: 0.0508
# average_iou: 0.1663
## Testing Data:
#   mean_ap25: 0.2500
#   mean_ap50: 0.1000
#   mean_ap75: 0.0750
# average_iou: 0.1760

### PointNet 4D Segmentation @ 50k
## Training Data:
#   mean_ap25: 0.2070
#   mean_ap50: 0.0586
#   mean_ap75: 0.0254
# average_iou: 0.1381
## Testing Data:
#   mean_ap25: 0.2150
#   mean_ap50: 0.0800
#   mean_ap75: 0.0350
# average_iou: 0.1481

### Density-Based Seeded Region Growing @ 25k
## Training Data:
#   mean_ap25: 0.
#   mean_ap50: 0.
#   mean_ap75: 0.
# average_iou: 0.
## Testing Data:
#   mean_ap25: 0.6497
#   mean_ap50: 0.2930
#   mean_ap75: 0.0000
# average_iou: 0.3318

  mean_ap25: 0.6496815286624203
  mean_ap50: 0.2929936305732484
  mean_ap75: 0.0
average_iou: 0.33175912896258675
